In [1]:
instance_uri = "https://api.stage.sinopia.io/resource/c3905dd8-e349-49b4-9ad3-55baeae178eb"
# change the instance_uri to the URI of the instance that is linked to the work resource you want to serialize for Alma RDF/XML.
# add api key and region for posting the resource to Alma
# https://github.com/LD4P/ils-middleware/wiki/Alma-APIs-in-Airflow

alma_api_key = "your_api_key"
uri_region = "https://api-na.hosted.exlibrisgroup.com"

In [2]:
!pip install rdflib
!pip install lxml
!pip install requests

In [3]:
import requests
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import RDF
from lxml import etree as ET
from name_space.alma_ns import alma_namespaces
from copy import deepcopy


In [4]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)


<Graph identifier=N5df22ef357c94be7a638132c80a397c9 (<class 'rdflib.graph.Graph'>)>

In [5]:
 # Define the bf namespace
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
for prefix, url in alma_namespaces:
    work_graph.bind(prefix, url)
work_uri = instance_graph.value(
    subject=URIRef(instance_uri), predicate=bf.instanceOf
        )
work_uri = URIRef(work_uri)
# Explicitly state that work_uri is of type bf:Work
work_graph.add((work_uri, RDF.type, bf.Work))

# parse the work graph
work_graph.parse(work_uri)

# add the instance to the work graph
work_graph.add((work_uri, bf.hasInstance, URIRef(instance_uri)))

# serialize the work graph
bfwork_alma_xml = work_graph.serialize(format="pretty-xml", encoding="utf-8")
tree = ET.fromstring(bfwork_alma_xml)
print(ET.tostring(tree, pretty_print=True).decode("utf-8"))
# save the work graph to a file
with open("work_graph.xml", "wb") as f:
    f.write(bfwork_alma_xml)
    

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/085754a9-082b-476e-84b3-137425e3226a">
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="fab4e83d54b6a43b597f90c8a3916f9fbb1">
        <bf:mainTitle xml:lang="en">Testing relations in Alma</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="fab4e83d54b6a43b597f90c8a3916f9fbb2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no90020704"/>
      </bf:PrimaryContribution>
    </bf:contribution>
    <bf:originDate xml:lang="en">1948</bf:originDate>
    <bf:language>
      <bf:Language rdf:about="http://id.loc.gov/voc

In [6]:
from lxml import etree as ET
from copy import deepcopy

# Parse the XML file
tree = ET.parse('work_graph.xml')
work_graph = tree.getroot()

# Define namespaces
namespaces = {'bf': 'http://id.loc.gov/ontologies/bibframe/',
              'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
              'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'}

# Find all bf:Work elements
works = work_graph.xpath('//bf:Work', namespaces=namespaces)

for work in works:
    work_about = work.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about']

    # Find the bf:relatedTo element with the same rdf:resource attribute value
    related_to = work_graph.xpath(f'//bf:relatedTo[@rdf:resource="{work_about}"]', namespaces=namespaces)

    if related_to:
        # Remove the rdf:resource attribute from the bf:relatedTo element
        related_to[0].attrib.pop('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource', None)

        # Clone the bf:Work element and append it under the bf:relatedTo element
        cloned_work = deepcopy(work)
        related_to[0].append(cloned_work)

        # Remove the original bf:Work element that was cloned
        work.getparent().remove(work)

# Print the modified XML
print(ET.tostring(work_graph, pretty_print=True).decode())
# save the modified XML to a file
with open('bfwork_alma.xml', 'wb') as f:
    f.write(ET.tostring(work_graph, pretty_print=True))
    f.close()

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/085754a9-082b-476e-84b3-137425e3226a">
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="fab4e83d54b6a43b597f90c8a3916f9fbb1">
        <bf:mainTitle xml:lang="en">Testing relations in Alma</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="fab4e83d54b6a43b597f90c8a3916f9fbb2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no90020704"/>
      </bf:PrimaryContribution>
    </bf:contribution>
    <bf:originDate xml:lang="en">1948</bf:originDate>
    <bf:language>
      <bf:Language rdf:about="http://id.loc.gov/voc

In [7]:
# open the file and parse the XML
tree = ET.parse("bfwork_alma.xml")
xslt = ET.parse("xsl/normalize-work-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
alma_xml = transform(tree)
alma_xml = ET.tostring(
        alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("alma-work.xml", "wb") as f:
    f.write(alma_xml)
print(alma_xml.decode("utf-8"))

<bib>
  <record_format>lcbf_work</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/085754a9-082b-476e-84b3-137425e3226a" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="fab4e83d54b6a43b597f90c8a3916f9fbb1">
        <bf:mainTitle xml:lang="en">Testing relations in Alma</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="fab4e83d54b6a43b597f90c8a3916f9fbb2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no90020704"/>
   

In [ ]:
# handle 400, an update to the record in Alma
def parse_400(result):
    xml_response = ET.fromstring(result)
    xslt = ET.parse("xsl/put_mms_id.xsl")
    transform = ET.XSLT(xslt)
    result_tree = transform(xml_response)
    put_mms_id_str = str(result_tree)
    print(f"put_mms_id_str: {put_mms_id_str}")
    return put_mms_id_str

# post the work to Alma
def NewWorktoAlma():
    with open("alma-work.xml", "rb") as f:
        data = f.read()

        alma_uri = (
            uri_region
            + "/almaws/v1/bibs?"
            + "from_nz_mms_id=&from_cz_mms_id=&normalization=&validate=false"
            + "&override_warning=true&check_match=false&import_profile=&apikey="
            + alma_api_key
        )
        # post to alma
        alma_result = requests.post(
            alma_uri,
            headers={
                "Content-Type": "application/xml; charset=utf-8",
                "Accept": "application/xml",
                "x-api-key": alma_api_key,
            },
            data=data,
        )
        print(f"alma result: {alma_result.status_code}\n{alma_result.text}")
        result = alma_result.content
        status = alma_result.status_code
        if status == 200:
            xml_response = ET.fromstring(result)
            mms_id = xml_response.xpath("//mms_id/text()")
            print(f"Created record {mms_id}")
        elif status == 400:
            # run xslt on the result in case the response is 400 and we need to update the record
            put_mms_id_str = parse_400(result)
            alma_update_uri = (
                uri_region
                + "/almaws/v1/bibs/"
                + put_mms_id_str
                + "?normalization=&validate=false&override_warning=true"
                + "&override_lock=true&stale_version_check=false&cataloger_level=&check_match=false"
                + "&apikey="
                + alma_api_key
            )
            putWorkToAlma(
                alma_update_uri,
                data,
            )
        else:
            raise Exception(f"Unexpected status code from Alma API: {status}")

# update the instance in Alma
def putWorkToAlma(
    alma_update_uri,
    data,
):
    put_update = requests.put(
        alma_update_uri,
        headers={
            "Content-Type": "application/xml; charset=UTF-8",
            "Accept": "application/xml",
        },
        data=data,
    )
    print(f"put update: {put_update.status_code}\n{put_update.text}")
    put_update_status = put_update.status_code
    result = put_update.content
    xml_response = ET.fromstring(result)
    put_mms_id = xml_response.xpath("//mms_id/text()")
    match put_update_status:
        case 200:
            print(f"Updated record {put_mms_id}")
        case 500:
            raise Exception(f"Internal server error from Alma API: {put_update_status}")
        case _:
            raise Exception(
                f"Unexpected status code from Alma API: {put_update_status}"
            )

# Call the function
NewWorktoAlma()